In [2]:
import os
from difflib import SequenceMatcher
from tqdm import tqdm

INPUT_RAW = '/home/burak/Desktop/IM/im/data/tw_data/PDTB_Annotations_20200205__/raw'
ANNOTATED_FOLDER = '/home/burak/Desktop/IM/im/data/tw_data/PDTB_Annotations_20200205__/ann/raw'
CLEANED_ANNOTATED_FOLDER = '/home/burak/Desktop/IM/im/data/tw_data/PDTB_Annotations_20200205__/cleaned/ann'

In [3]:
def clean_file(input_raw, input_raw_annotation, output_annotation_clean):
    with open(input_raw, 'r', encoding='utf8') as file:
        file_content = file.read()
        new_thread = []
        for line in file_content.split('\n'):
            if (len(line) == 0):
                break
            line_pieces = line.split('\t')
            if len(line_pieces) > 1:
                handle = line_pieces[0]
                # +1 is for the tab.
                new_thread.append(line_pieces[1])
            else:
                new_thread.append(line)
            new_thread.append('\n')
        new_thread = ''.join(new_thread)
        
        # autojunk is too optimal
        sm = SequenceMatcher(None, file_content, new_thread, autojunk=False)
        ops = sm.get_opcodes()
        with open(input_raw_annotation, encoding='utf8') as ann_file:
            ann_file_content = ann_file.read()
            new_content = []
            _start = 0
            _end = 1
            _in_number = False
            for i, ann_char  in enumerate(ann_file_content):
                if not _in_number:
                    if ann_char.isdigit():
                        _start = i
                        _in_number = True
                    else:
                        new_content.append(ann_char)
                else:
                    if not ann_char.isdigit():
                        _end = i
                        _in_number = False
                        if not ann_file_content[_start - 1].isalpha():
                            num = int(ann_file_content[_start:_end])
                            actual_num = len(file_content.encode('utf-16le')[:(num * 2)].decode('utf-16le'))
                            for tag, i1, i2, j1, j2 in sm.get_opcodes():
                                if actual_num >= i1 and actual_num <= i2:
                                    if tag == 'equal':
                                        new_num = num - (i1 - j1)
                                        break
                            new_content.append(str(new_num))
                            new_content.append(ann_char)
                        _start = _end + 1
                        continue
            with open(output_annotation_clean, 'w', encoding='utf8') as cleaned_ann:
                cleaned_ann.write(''.join(new_content))

In [7]:
files = os.listdir('data/tw_data/PDTB_Annotations_20200205__/raw/')
for file_name in tqdm(files):
    try:
        clean_file(
            INPUT_RAW + '/' + file_name,
            ANNOTATED_FOLDER + '/' + file_name,
            CLEANED_ANNOTATED_FOLDER + '/' + file_name
        )
    except:
        print(file_name)

 50%|█████     | 93/185 [00:28<00:18,  5.08it/s]

093_950279658192347136.branch49.txt.username_text_tabseparated


 66%|██████▋   | 123/185 [00:32<00:11,  5.31it/s]

095_950296746743169024.branch3.txt.username_text_tabseparated


100%|██████████| 185/185 [00:54<00:00,  3.37it/s]


In [7]:
clean_file('030_948126677648932869.branch4.txt.username_text_tabseparated')